<a href="https://colab.research.google.com/github/hmahadik/jupyter_notebooks/blob/master/Object_detection_for_pretrained_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2018 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

In [ ]:
#@title


# Object detection

<table align="left"><td>
  <a target="_blank"  href="https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/object_detection.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab
  </a>
</td><td>
  <a target="_blank"  href="https://github.com/tensorflow/hub/blob/master/examples/colab/object_detection.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
</td></table>


This Colab demonstrates use of a TF-Hub module trained to perform object detection.

## Installs

In [ ]:
#@title All installs { display-mode: "form" }
# Currently %tensorflow_version 2.x installs beta1, which doesn't work here.
# %tensorflow_version can likely be used after 2.0rc0  

!git clone --quiet https://github.com/tensorflow/models.git
!cp -R models/research/object_detection/ object_detection/
!protoc object_detection/protos/*.proto --python_out=.
!git clone --quiet https://github.com/zamblauskas/oidv4-toolkit-tfrecord-generator.git
!git clone --quiet https://github.com/EscVM/OIDv4_ToolKit.git
!pip3 install -q -r OIDv4_ToolKit/requirements.txt
!pip install tf_slim

## Imports and function definitions


In [ ]:
#@title All imports { display-mode: "form" }
from __future__ import absolute_import, division, print_function, unicode_literals
# For running inference on the TF-Hub module.
import tensorflow as tf

#tf.enable_eager_execution()
import tensorflow_hub as hub

# For downloading the image.
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

# For drawing onto the image.
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

# For annotation functions
import pandas as pd
import os

# For measuring the inference time.
import time
import datetime
import sys
import shutil

sys.path.append("..")
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

from object_detection.utils import ops as utils_ops
from object_detection.utils import object_detection_evaluation
from object_detection.utils import per_image_evaluation

from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util

# Check available GPU devices.
print("The following GPU devices are available: %s" % tf.test.gpu_device_name())
# This is needed to display the images.
%matplotlib inline
from shutil import copyfile

## All directories (Set by yourself)

In [ ]:
#@title ##Fill in part
#for the classes we actually use
#Please only enter the classes in mscoco dataset
#classes_set = ['Box', 'Man', 'Suitcase', 'Handbag', 'Backpack']
#for the classes we are looking for
#@markdown ###Please choose from mscoco dataset for classes
classes_set2 = ['Person', 'Suitcase', 'Handbag', 'Backpack'] #@param {type:"raw"}
#@markdown ---
#number of images for training
num_train = 150 #@param {type:"integer"}
num_val = 50 #@param {type:"integer"}

#name of pipeline file
pipeline_file = 'ssd_mobilenet_v2_coco.config' #@param {type:"string"}
label_file_name = 'oid_bbox_trainable_label_map.pbtxt' #@param {type:"string"}
#where the model is saved
MODEL_DIR = '/content/model_trained' #@param {type:"string"}
#number of steps
TRAIN_STEP = 50000 #@param {type:"integer"}
EVAL_STEP = 50 #@param {type:"integer"}

batch_size = 8 #@param {type:"integer"}
num_steps = 50000 #@param {type:"integer"}

load_all_ckpt = False #@param {type:"boolean"}

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8) #@param {type:"raw"}

MODEL_NAME_CHECK = 'ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03' #@param ['ssd_mobilenet_v1_coco_2017_11_17', 'ssd_mobilenet_v2_coco_2018_03_29', 'faster_rcnn_resnet101_coco_2018_01_28','ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03'] {type:"string"}
MODEL_NAME_TRAIN = 'ssd_mobilenet_v2_coco_2018_03_29' #@param {type:"string"}
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/' #@param {type:"string"}

output_freeze_directory = '/content/out_graph/'

In [ ]:
#@title Other directories { display-mode: "form" }

num_classes = len(classes_set2)
class_description_path = '/content/OID/csv_folder/class-descriptions-boxable.csv'
annotation_path_train = '/content/OID/csv_folder/train-annotations-bbox.csv'
annotation_path_val = '/content/OID/csv_folder/validation-annotations-bbox.csv'
fail_dir = '/content/failed_image'
model_dir = '/content/model_trained'
class_dir = '/content/OIDv4_ToolKit/classes.txt'
tf_dir = '/content/tfrecord'

val_record_fname = os.path.join(tf_dir, 'val.tfrecord')
train_record_fname = os.path.join(tf_dir, 'train.tfrecord')

## Download images

In [ ]:
#@title Download images from OID! { display-mode: "form" }

seperator = ' '
print(seperator.join(classes_set2))
class_str = seperator.join(classes_set2)
seperator_ul = '_'
class_str_ul = seperator_ul.join(classes_set2)

image_dir_train = '/content/OID/Dataset/train/'+class_str_ul+'/'
image_dir_val = '/content/OID/Dataset/validation/'+class_str_ul+'/'
%cd /content/
!python3 OIDv4_ToolKit/main.py downloader -y --classes {class_str} --type_csv train --limit {num_train} --multiclasses 1
!python3 OIDv4_ToolKit/main.py downloader -y --classes {class_str} --type_csv validation --limit {num_val} --multiclasses 1

## Helper functions for downloading images and for visualization.

Visualization code adapted from [TF object detection API](https://github.com/tensorflow/models/blob/master/research/object_detection/utils/visualization_utils.py) for the simplest required functionality.

In [ ]:
#@title make_dir { display-mode: "form" }
def make_dir(dir_name):
    try:
        os.mkdir(dir_name)
    except OSError:
        print ("Creation of the directory %s failed" % dir_name)
    else:
        print ("Successfully created the directory %s " % dir_name)

In [ ]:
#@title Read annotations. { display-mode: "form" }
#read annotation for certain image from annotation file
def read_annotation(classes_set, 
                    class_description_path, 
                    annotation_path, 
                    image_dir):
  
  classes = list(filter(None, classes_set))
  classes = {name: idx + 1 for idx, name in enumerate(classes)}
  class_descriptions = {row[0]: row[1] for _, row in pd.read_csv(class_description_path, header=None).iterrows()}
  annotations = pd.read_csv(annotation_path)
  annotations['LabelName'] = annotations['LabelName'].map(lambda n: class_descriptions[n])
  annotations = annotations.groupby('ImageID')             
  #images = tf.io.gfile.glob(image_dir + '/*.jpg')
  images = tf.io.gfile.glob(os.path.join(image_dir, '*.jpg'))
  images = map(lambda i: (os.path.basename(i).split('.jpg')[0], i), images)
  images = dict(images)           

  image_pathes = []               
  xmins = []
  xmaxs = []
  ymins = []
  ymaxs = []
  boxes = []
  classes_text = []
  classes_int = []  
  is_group = []
  is_difficult = []
  count = 0               
  for image_id, path in images.items():
        img_width, img_height = Image.open(path).size
        #img_data = tf.gfile.GFile(path, 'rb').read()
        image_annotations = annotations.get_group(image_id)
        for _, row in image_annotations.loc[image_annotations['LabelName'].isin(classes.keys())].iterrows():
            image_pathes.append(row['ImageID'])
            xmins.append(row['XMin'])
            xmaxs.append(row['XMax'])
            ymins.append(row['YMin'])
            ymaxs.append(row['YMax'])
            classes_text.append(row['LabelName'].encode('utf8'))
            classes_int.append(classes[row['LabelName']])
            is_group.append(row['IsGroupOf'])
            is_difficult.append(0)
  boxes = np.concatenate(([xmins], [ymins], [xmaxs], [ymaxs]),0)
  boxes = np.transpose(boxes)
  #image_pathes = np.transpose(image_pathes)
  classes_text = np.array(classes_text)
  classes_int = np.array(classes_int)
  is_group = np.array(is_group)
  is_difficult = np.array(is_difficult)
  #boxes = [np.reshape(xmins, (-1, 1)),np.reshape(xmaxs, (-1, 1)),np.reshape(ymins, (-1, 1)),np.reshape(ymaxs, (-1, 1))]
  return image_pathes, boxes, classes_text, classes_int, is_group, is_difficult
                 


In [ ]:
#@title Write tfrecord { display-mode: "form" }
def write_tf(classes_file, output_file, class_descriptions_file, annotations_file, image_dir):
    classes = list(filter(None, open(classes_file).read().split('\n')))
    classes = {name: idx + 1 for idx, name in enumerate(classes)}
    #print(f'Classes: {classes}')
    #print(f'Classes: {classes}')
    class_descriptions = {row[0]: row[1] for _, row in pd.read_csv(class_descriptions_file, header=None).iterrows()}

    annotations = pd.read_csv(annotations_file)
    annotations['LabelName'] = annotations['LabelName'].map(lambda n: class_descriptions[n])
    annotations = annotations.groupby('ImageID')

    images = tf.io.gfile.glob(image_dir + '*.jpg')
    images = map(lambda i: (os.path.basename(i).split('.jpg')[0], i), images)
    images = dict(images)
    writer = tf.io.TFRecordWriter(output_file)
    for image_id, path in images.items():
        #print(image_id)
        img_width, img_height = Image.open(path).size
        img_data = tf.io.gfile.GFile(path, 'rb').read()

        xmins = []
        xmaxs = []
        ymins = []
        ymaxs = []
        classes_text = []
        classes_int = []

        image_annotations = annotations.get_group(image_id)
        for _, row in image_annotations.loc[image_annotations['LabelName'].isin(classes.keys())].iterrows():
            xmins.append(row['XMin'])
            xmaxs.append(row['XMax'])
            ymins.append(row['YMin'])
            ymaxs.append(row['YMax'])
            classes_text.append(row['LabelName'].encode('utf8'))
            classes_int.append(classes[row['LabelName']])

        tf_example = tf.train.Example(features=tf.train.Features(feature={
            'image/height': tf.train.Feature(int64_list=tf.train.Int64List(value=[img_height])),
            'image/width': tf.train.Feature(int64_list=tf.train.Int64List(value=[img_width])),
            'image/filename': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_id.encode('utf8')])),
            'image/source_id': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_id.encode('utf8')])),
            'image/encoded': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_data])),
            'image/format': tf.train.Feature(bytes_list=tf.train.BytesList(value=[b'jpg'])),
            'image/object/bbox/xmin': tf.train.Feature(float_list=tf.train.FloatList(value=xmins)),
            'image/object/bbox/xmax': tf.train.Feature(float_list=tf.train.FloatList(value=xmaxs)),
            'image/object/bbox/ymin': tf.train.Feature(float_list=tf.train.FloatList(value=ymins)),
            'image/object/bbox/ymax': tf.train.Feature(float_list=tf.train.FloatList(value=ymaxs)),
            'image/object/class/text': tf.train.Feature(bytes_list=tf.train.BytesList(value=classes_text)),
            'image/object/class/label': tf.train.Feature(int64_list=tf.train.Int64List(value=classes_int))
        }))

        writer.write(tf_example.SerializeToString())
        #print('.', end='')
    writer.close()
    print(" done")



In [ ]:
#@title load_image_into_numpy_array { display-mode: "form" }
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  try:
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)
  except:
    return []



In [ ]:
#@title run_inference_for_single_image { display-mode: "form" }
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}

      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[1], image.shape[2])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: image})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.int64)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
      
  return output_dict

In [ ]:
#@title run_filter_on_dir2 { display-mode: "form" }

def run_filter_on_dir2(img_dir, annotation_path):
  g_pathes_all, g_boxes_all, g_classes_all, g_classes_int_all, is_g_all, is_d_all = read_annotation(classes_set2, class_description_path, annotation_path, img_dir)
  img_name_list = [x.split('.')[0] for x in os.listdir(img_dir)]
  for img_name in img_name_list:
    if img_name == 'Label':
        continue
    img_path_curr = os.path.join(img_dir, img_name)+'.jpg'
    image = Image.open(img_path_curr)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    if image_np == []:
      img_path_new = os.path.join(fail_dir, img_name)+'.jpg'
      shutil.move(img_path_curr, img_path_new) 
      continue
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np_expanded, detection_graph)

    idx_new = []
    for count, class_int in enumerate(output_dict['detection_classes']):
      #print(class_int)
      class_name = list(label_dict_coco.keys())[list(label_dict_coco.values()).index(class_int)]
      if class_name in label_dict.keys():
        #print(class_name)
        idx_new.append(count)
        output_dict['detection_classes'][count] = label_dict[class_name]

    for item in output_dict:
      if item == 'num_detections':
        continue
      output_dict[item] = output_dict[item][idx_new]
    d_boxes = np.copy(output_dict["detection_boxes"])
    #d_boxes_copy = output_dict["detection_boxes"]
    #print(output_dict["detection_boxes"])
    #print(d_boxes)
    #format to match the annotation 
    temp = np.copy(d_boxes[:, 0])
    d_boxes[:, 0] = d_boxes[:, 1]
    d_boxes[:, 1] = temp
    temp = np.copy(d_boxes[:, 2])
    d_boxes[:, 2] = d_boxes[:, 3]
    d_boxes[:, 3] = temp
    d_scores = output_dict["detection_scores"]
    d_class_int = output_dict["detection_classes"]
    idx_new = []
    for i in range(len(g_pathes_all)):
      if g_pathes_all[i] == img_name:
        idx_new.append(i)
    g_boxes = np.array(g_boxes_all[idx_new])
    g_classes = np.array(g_classes_all[idx_new])
    is_g = np.array(is_g_all[idx_new])
    is_d = np.array(is_d_all[idx_new])
    g_classes_int = np.array(g_classes_int_all[idx_new])
    is_g = np.array(is_g,dtype=bool)
    evaluator = per_image_evaluation.PerImageEvaluation(num_classes+1)  
    #print(d_class_int)
    rearrange_class_idx(classes, label_dict, d_class_int)
    #print(d_class_int)
    #print(g_classes_int)
    scores, tp_fp_labels, is_class_correctly_detected_in_image = evaluator.compute_object_detection_metrics(d_boxes, d_scores, d_class_int, g_boxes, g_classes_int, is_d, is_g)

    detected_right_num = 0
    for item_array in tp_fp_labels:
      detected_right_num = detected_right_num+sum(item_array) 
    num_ground_truth = np.shape(g_boxes)[0]
    print(detected_right_num)
    print(num_ground_truth)  
    if detected_right_num == num_ground_truth and detected_right_num != 0:
      print("image " + img_name + " passed!")
    else:
      print("image " + img_name + " failed! Image Removed!")
      img_path_new = os.path.join(fail_dir, img_name)+'.jpg'
      shutil.move(img_path_curr, img_path_new) 

    # Visualization of the results of a detection.
'''    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        #d_boxes_copy,
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=8)
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image_np)'''

##Using pretrained model

In [ ]:
# What model to download.

MODEL_FILE = MODEL_NAME_CHECK + '.tar.gz'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME_CHECK + '/frozen_inference_graph.pb'


In [ ]:
import re
from object_detection.utils import label_map_util

PATH_TO_LABELS = os.path.join('/content/object_detection/data/', label_file_name)
dst_label = os.path.join('/content/', label_file_name)
copyfile(PATH_TO_LABELS, dst_label)

label_map_int = label_map_util.load_labelmap(dst_label)
for item in label_map_int.item:
    if item.display_name not in classes_set2:
        item.Clear()


input_txt = str(label_map_int)
input_txt = input_txt.replace("item {\n}\n", "")
#print(input_txt)
with open(dst_label, 'w') as f:
    s = f.write(input_txt)

In [ ]:
import six.moves.urllib as urllib
import tarfile
%cd /content/
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  tar_file.extract(file, os.getcwd())

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.compat.v1.GraphDef()
  with tf.io.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [ ]:
label_dict = label_map_util.get_label_map_dict(dst_label, use_display_name=True)
label_dict = {k.lower():v for k, v in label_dict.items()}
print(label_dict)


In [ ]:
label_map = label_map_util.load_labelmap(dst_label)
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=num_classes, use_display_name=True)
category_index = label_map_util.create_category_index(categories)
label_map_coco_path = '/content/object_detection/data/mscoco_label_map.pbtxt'
label_dict_coco = label_map_util.get_label_map_dict(label_map_coco_path, use_display_name=True)
label_dict_coco = {k.lower():v for k, v in label_dict_coco.items()}
#print(label_dict_coco)

In [ ]:
classes = list(filter(None, classes_set2))
classes = {name.lower(): idx + 1 for idx, name in enumerate(classes)}
print(classes)

def rearrange_class_idx(class_dict, dataset_dict, class_list):
  for idx, class_id in enumerate(class_list):
    #print(class_id)
    class_name = list(dataset_dict.keys())[list(dataset_dict.values()).index(class_id)]
    #print(class_name)
    class_list[idx] = class_dict[class_name]



In [ ]:
#@title make directories { display-mode: "form" }

# make directories
make_dir(fail_dir)
make_dir(tf_dir)

In [ ]:
#@title Run compare! { display-mode: "form" }
print("Start filtering training set...")
run_filter_on_dir2(image_dir_train, annotation_path_train)
print("Done!")
print("Start filtering validation set...")
run_filter_on_dir2(image_dir_val, annotation_path_val)
print("Done!")

In [ ]:
#@title Modify config file { display-mode: "form" }

import re

pipeline_fname = os.path.join('/content/models/research/object_detection/samples/configs/', pipeline_file)
assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)
dst_ppl = os.path.join('/content/', pipeline_file)
copyfile(pipeline_fname, dst_ppl)

DEST_DIR = os.path.join('/content/', MODEL_NAME_TRAIN)
fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")

label_map_int = label_map_util.load_labelmap(dst_label)
num_classes = label_map_util.get_max_label_map_index(label_map_int)

num_image = len(os.listdir(image_dir_val)) -1
print(num_classes)


with open(dst_ppl) as f:
    s = f.read()
with open(dst_ppl, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(val_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(dst_label), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    
    # Set validating image number
    s = re.sub('num_examples: [0-9]+',
               'num_examples: {}'.format(num_image), s)

    if load_all_ckpt == True:
        s = re.sub('fine_tune_checkpoint_type:  "detection"\n', 
                   'fine_tune_checkpoint_type:  "detection"\n  load_all_detection_checkpoint_vars: true\n', s)

    f.write(s)

## Run Tensorboard(Optional)

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

In [ ]:
LOG_DIR = model_dir
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [ ]:
get_ipython().system_raw('./ngrok http 6006 &')

### Get Tensorboard link

In [ ]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

## Make tf record files



In [ ]:
#write in class file
with open(class_dir, "w") as file:
  file.writelines(["%s\n" % item  for item in classes_set2])


In [ ]:
output_file='/content/tfrecord/train.tfrecord'
classes_file = class_dir
class_descriptions_file = class_description_path
annotations_file = annotation_path_train
image_dir = image_dir_train

write_tf(classes_file, output_file, class_descriptions_file, annotations_file, image_dir)

output_file='/content/tfrecord/val.tfrecord'
classes_file = class_dir
class_descriptions_file = class_description_path
annotations_file = annotation_path_val
image_dir = image_dir_val

write_tf(classes_file, output_file, class_descriptions_file, annotations_file, image_dir)

##download pretrained model

In [ ]:
#%cd /content/

#!wget http://download.tensorflow.org/models/object_detection/${MODEL_NAME_TRAIN}.tar.gz
#!tar -xzvf ${MODEL_NAME_TRAIN}.tar.gz

MODEL_FILE = MODEL_NAME_TRAIN + '.tar.gz'
print(DOWNLOAD_BASE + MODEL_FILE)
%cd /content/
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  tar_file.extract(file, os.getcwd())

In [ ]:

!cat $dst_ppl

##train

In [ ]:
%cd /content
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib

!pip install -q pycocotools

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

!python object_detection/builders/model_builder_test.py

In [ ]:
%cd /content/models/research
!export PYTHONPATH=$PYTHONPATH:/content/models/research:/content/models/research/slim
!rm -rf $MODEL_DIR
!mkdir $MODEL_DIR
!python3 /content/object_detection/model_main.py \
    --pipeline_config_path=$dst_ppl \
    --model_dir=$MODEL_DIR \
    --num_train_steps=$TRAIN_STEP \
    --num_eval_steps=$EVAL_STEP \
    --alsologtostderr \
    --sample_1_of_n_eval_examples=1


## Exporting a Trained Inference Graph
Once your training job is complete, you need to extract the newly trained inference graph, which will be later used to perform the object detection. This can be done as follows:

In [ ]:
import re
import numpy as np

lst = os.listdir(model_dir)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)
!rm -rf $output_freeze_directory
!mkdir $output_freeze_directory
!python /content/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path=$dst_ppl \
    --output_directory=$output_freeze_directory \
    --trained_checkpoint_prefix=$last_model_path \
    --run_once

In [ ]:
!ls {output_freeze_directory}

In [ ]:
import os

pb_fname = os.path.join(os.path.abspath(output_freeze_directory), "frozen_inference_graph.pb")
assert os.path.isfile(pb_fname), '`{}` not exist'.format(pb_fname)

In [ ]:
print(pb_fname)

In [ ]:
from google.colab import files
files.download(pb_fname)